# Đề thi Giữa kỳ Hệ khuyến nghị 2025.

### Khoa học dữ liệu IUH. Ngày 16/10/2025. Thời gian làm bài: 90 phút. Ca 1.

**Bài 1.** (*2 điểm*) Cho file dữ liệu *data.csv* về $25$ bài hát được $400$ users đã rate trong tháng qua, điểm từ $1$ đến $5$. Cấu trúc file: *id user, id bài hát, điểm rating*. Gọi $a$ là tổng hai chữ số cuối trong MSSV của anh/chị.

1) Sử dụng kỹ thuật tổng hợp 2 thông tin: số rating kết hợp rating trung bình, hãy đề xuất ra top $10$ bài hát nên nghe cho một user cold-start.
2) Giả sử có một user đã nghe bài hát STT $a$. Sử dụng kỹ thuật đếm số cặp bài hát được nghe bởi cùng user, hãy đề xuất ra top 5 bài hát mà user này có thể cũng muốn nghe theo xu hướng.

In [2]:
# 1.1
import pandas as pd
import numpy as np

# Đọc file (giả sử tên file là "songs.csv")
df = pd.read_csv("T:\GK_he khuyen nghi_ca 1\data.csv")
df.columns = ['user_id', 'song_id', 'rating']

# Tính số lượng rating và trung bình rating cho mỗi bài hát
song_stats = df.groupby('song_id')['rating'].agg(['count', 'mean']).reset_index()

# Tính trung bình rating toàn hệ thống
global_mean = df['rating'].mean()

# Hệ số điều chỉnh để tránh ưu tiên các bài có ít rating
k = 5

# Tính điểm tổng hợp theo công thức Bayesian Average
song_stats['score'] = (song_stats['count'] / (song_stats['count'] + k)) * song_stats['mean'] \
                      + (k / (song_stats['count'] + k)) * global_mean

# Sắp xếp giảm dần theo điểm tổng hợp
top10_cold_start = song_stats.sort_values('score', ascending=False).head(10)

print("=== 🧠 GỢI Ý CHO USER MỚI (COLD-START USER) ===")
print(top10_cold_start[['song_id', 'count', 'mean', 'score']])
print()

=== 🧠 GỢI Ý CHO USER MỚI (COLD-START USER) ===
    song_id  count      mean     score
3         4     70  3.400000  3.371347
2         3     69  3.347826  3.322311
9        10     64  3.296875  3.273203
0         1     70  3.100000  3.091347
24       25     71  3.098592  3.090145
23       24     61  3.098361  3.088652
6         7     71  3.084507  3.076987
17       18     64  3.078125  3.070305
10       11     57  3.052632  3.045984
11       12     67  3.044776  3.039597



In [3]:
#1.2

# Tạo ma trận user-song: mỗi ô = 7 nếu user nghe bài hát đó
pivot = pd.crosstab(df['user_id'], df['song_id'])

# Tính số lần 2 bài hát được nghe cùng nhau
co_occurrence = pivot.T.dot(pivot)

# Xóa chính nó (để không tự so sánh chính mình)
np.fill_diagonal(co_occurrence.values, 0)

# Lấy hàng tương ứng với bài hát STT = 7 (nếu tồn tại)
if 7 in co_occurrence.index:
    song1_related = co_occurrence.loc[7].sort_values(ascending=False).head(5)
    print("=== 🎧 GỢI Ý CHO USER ĐÃ NGHE BÀI HÁT STT = 7 ===")
    print(song1_related)
else:
    print("⚠️ Bài hát STT = 7 chưa xuất hiện trong dữ liệu, không thể tính co-occurrence.")
print()

=== 🎧 GỢI Ý CHO USER ĐÃ NGHE BÀI HÁT STT = 7 ===
song_id
13    18
23    18
5     18
9     18
24    17
Name: 7, dtype: int64



**Bài 2.** (*2.5 điểm*) Sử dụng dataset movie_lens trong đó có file *movies.csv* mô tả tên films và các thể loại (một phim có thể thuộc nhiều thể loại).

1. Đọc dữ liệu và lọc lại $200$ films có nhiều thể loại nhất. Ngoài ra, hãy cho biết tổng cộng có bao nhiêu thể loại khác nhau.
2. Xét film $X$ ở vị trí $25$ trong danh sách nêu trên, tính ra top $10$ films tương đồng với film $X$ này nhất bằng *độ tương đồng Jaccard*.
3. Một user $Y$ đã xem các bộ phim có STT trong tập hợp $ID$, hãy xây dựng profile cơ bản cho user này và khuyến nghị ra top $10$ films mà user $Y$ này có thể muốn xem.

In [4]:
# 2.1
import pandas as pd

# Đọc dữ liệu từ file movies.csv
movies = pd.read_csv('T:\GK_he khuyen nghi_ca 1\movies.csv')

# Tách các thể loại phim
movies['genres'] = movies['genres'].str.split('|')

# Đếm số thể loại của mỗi phim
movies['num_genres'] = movies['genres'].apply(len)

# Lọc ra 200 phim có nhiều thể loại nhất
top_200_movies = movies.nlargest(200, 'num_genres')
print(top_200_movies[['movieId', 'title', 'num_genres', 'genres']])

      movieId                                              title  num_genres  \
7441    81132                                      Rubber (2010)          10   
5556    26701  Patlabor: The Movie (Kidô keisatsu patorebâ: T...           8   
1390     1907                                       Mulan (1998)           7   
2250     2987                    Who Framed Roger Rabbit? (1988)           7   
3460     4719                               Osmosis Jones (2001)           7   
...       ...                                                ...         ...   
5379     8968                            After the Sunset (2004)           5   
5382     8972                           National Treasure (2004)           5   
5470    26183  Asterix and Cleopatra (Astérix et Cléopâtre) (...           5   
5471    26184      Diamond Arm, The (Brilliantovaya ruka) (1968)           5   
5539    26614                        Bourne Identity, The (1988)           5   

                                       

In [5]:
# 2.2
# Hàm tính độ tương đồng Jaccard giữa hai bộ phim
def jaccard_similarity(genres1, genres2):
    set1 = set(genres1)
    set2 = set(genres2)
    intersection = len(set1 & set2)
    union = len(set1 | set2)
    return intersection / union if union != 0 else 0

# Giả sử chúng ta xét phim X có chỉ số là movieId_X
movieId_X = top_200_movies.iloc[25]['movieId']
genres_X = top_200_movies[top_200_movies['movieId'] == movieId_X]['genres'].values[0]

# Tính độ tương đồng Jaccard với các phim khác
similarities = []
for _, row in top_200_movies.iterrows():
    if row['movieId'] != movieId_X:  # Loại bỏ chính phim X
        similarity = jaccard_similarity(genres_X, row['genres'])
        similarities.append((row['movieId'], similarity, row['title']))

# Sắp xếp theo độ tương đồng giảm dần và lấy top 10 phim
top_10_similar = sorted(similarities, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 phim tương đồng với phim X:")
for movie in top_10_similar:
    print(f"{movie[2]} (ID: {movie[0]}), Jaccard similarity: {movie[1]:.4f}")

Top 10 phim tương đồng với phim X:
Cirque du Freak: The Vampire's Assistant (2009) (ID: 72165), Jaccard similarity: 1.0000
Army of Darkness (1993) (ID: 1215), Jaccard similarity: 0.8333
Dragonheart 2: A New Beginning (2000) (ID: 117646), Jaccard similarity: 0.7143
Maximum Ride (2016) (ID: 164226), Jaccard similarity: 0.7143
True Lies (1994) (ID: 380), Jaccard similarity: 0.5714
Princess Bride, The (1987) (ID: 1197), Jaccard similarity: 0.5714
Goonies, The (1985) (ID: 2005), Jaccard similarity: 0.5714
Mighty Joe Young (1998) (ID: 2429), Jaccard similarity: 0.5714
Golden Child, The (1986) (ID: 2735), Jaccard similarity: 0.5714
End of Days (1999) (ID: 3113), Jaccard similarity: 0.5714


In [6]:
# 2.3
# Các phim user Y đã xem
user_movies = [2,6,5,9,1]

# Lấy thể loại của các phim mà user đã xem
user_genres = []
for movieId in user_movies:
    # Kiểm tra xem movieId có trong top_100_movies không
    movie_genres = top_200_movies[top_200_movies['movieId'] == movieId]['genres']
    if not movie_genres.empty:
        genres = movie_genres.values[0]
        user_genres.extend(genres)

# Tính tần suất xuất hiện của các thể loại trong profile của user Y
from collections import Counter
user_profile = Counter(user_genres)

# Tính độ tương đồng Jaccard giữa profile của user và các phim khác
user_profile_set = set(user_profile.keys())  # Các thể loại trong profile của user
recommendations = []
for _, row in top_200_movies.iterrows():
    movie_genres_set = set(row['genres'])  # Các thể loại của mỗi phim
    intersection = len(user_profile_set & movie_genres_set)
    union = len(user_profile_set | movie_genres_set)
    similarity = intersection / union if union != 0 else 0
    recommendations.append((row['movieId'], similarity, row['title']))

# Sắp xếp theo độ tương đồng giảm dần và lấy top 10 phim
top_10_recommendations = sorted(recommendations, key=lambda x: x[1], reverse=True)[:10]
print("Top 10 phim khuyến nghị cho user Y:")
for movie in top_10_recommendations:
    print(f"{movie[2]} (ID: {movie[0]}), Jaccard similarity: {movie[1]:.4f}")

Top 10 phim khuyến nghị cho user Y:
Toy Story (1995) (ID: 1), Jaccard similarity: 1.0000
Antz (1998) (ID: 2294), Jaccard similarity: 1.0000
Toy Story 2 (1999) (ID: 3114), Jaccard similarity: 1.0000
Adventures of Rocky and Bullwinkle, The (2000) (ID: 3754), Jaccard similarity: 1.0000
Emperor's New Groove, The (2000) (ID: 4016), Jaccard similarity: 1.0000
Monsters, Inc. (2001) (ID: 4886), Jaccard similarity: 1.0000
Space Jam (1996) (ID: 673), Jaccard similarity: 0.8333
Shrek (2001) (ID: 4306), Jaccard similarity: 0.8333
Twelve Tasks of Asterix, The (Les douze travaux d'Astérix) (1976) (ID: 26340), Jaccard similarity: 0.8333
Valiant (2005) (ID: 36397), Jaccard similarity: 0.8333


**Bài 3.** (*2.5 điểm*) Cho một bảng *utility matrix* giữa 8 user và 5 item như bên dưới:

|Rating     | $I_1$    | $I_2$  | $I_3$   |  $I_4$  | $I_5$  |
|----------|----------|--------|---------|---------|---------|
| $U_1$| 4    | 3      | 4    |   ?  |  5|
| $U_2$| 2    | 3      | 1    |   5  |  3|
| $U_3$| 5    | 4      |  3  |    4  |  2| 
| $U_4$|   4  |   2    | 2    |   5  |  4|
| $U_5$|   3  |   4    | ?    |   3  |  5|
| $U_6$|   5  |   3    | 5    |   5  |  1|
| $U_7$|   5  |   5    | 5    |   4  |  5|
| $U_8$|   1  |   5    | 2    |   2  |  4|

1. Dự đoán hai missing rating $r_{U_1, I_4}$ và $r_{U_5, I_2}$ bằng cách dùng *độ tương đồng Pearson*.
2. Câu hỏi tương tự trên nhưng chuẩn hoá bảng theo dạng *center-mean* và sau đó dùng *độ tương đồng cosine*.
3. Một người thực hiện dự đoán theo kiểu tuần tự, tức là dự đoán xong missing rating thứ $1$ thì điền rating đó vào bảng hiện tại rồi mới thực hiện tiếp cho missing rating thứ $2$. Anh chị hãy thực hiện cách làm này cho một trong hai ý 1) hoặc 2) và so sánh kết quả thu được. Hỏi cách làm này nói chung sẽ có các ưu nhược điểm gì?

In [7]:
# 3.1. Dự đoán missing rating bằng độ tương đồng Pearson
import numpy as np
import pandas as pd
from scipy.stats import pearsonr
from sklearn.metrics.pairwise import cosine_similarity

# Tạo utility matrix
data = {
    'I1': [4, 2, 5, 4, 3, 5, 5, 1],
    'I2': [3, 3, 4, 2, 4, 3, 5, 5],
    'I3': [4, 1, 3, 2, np.nan, 5, 5, 2],
    'I4': [np.nan, 5, 4, 5, 3, 5, 4, 2],
    'I5': [5, 3, 2, 4, 5, 1, 5, 4]
}

utility_matrix = pd.DataFrame(data, index=['U1', 'U2', 'U3', 'U4', 'U5', 'U6', 'U7', 'U8'])
print("Utility Matrix:")
print(utility_matrix)

# Hàm tính độ tương đồng Pearson giữa hai user
def pearson_similarity(user1, user2):
    common_items = user1.notna() & user2.notna()
    if common_items.sum() < 2:  # Cần ít nhất 2 điểm chung để tính Pearson
        return 0
    return pearsonr(user1[common_items], user2[common_items])[0]

def predict_pearson(user_id, item_id):
    user = utility_matrix.loc[user_id]
    similarities = []
    ratings = []
    
    for other_user_id in utility_matrix.index:
        if other_user_id != user_id and not pd.isna(utility_matrix.loc[other_user_id, item_id]):
            sim = pearson_similarity(user, utility_matrix.loc[other_user_id])
            similarities.append(sim)
            ratings.append(utility_matrix.loc[other_user_id, item_id])
    
    if sum(np.abs(similarities)) == 0:
        return np.nan  # Không thể dự đoán nếu không có độ tương đồng
    
    predicted_rating = np.dot(similarities, ratings) / sum(np.abs(similarities))
    return predicted_rating

predicted_r_U1_I4 = predict_pearson('U1', 'I4')
predicted_r_U5_I2 = predict_pearson('U5', 'I2')
print(f"\nPredicted rating r_{{U1, I4}} using Pearson: {predicted_r_U1_I4:.2f}")
print(f"Predicted rating r_{{U5, I2}} using Pearson: {predicted_r_U5_I2:.2f}")

Utility Matrix:
    I1  I2   I3   I4  I5
U1   4   3  4.0  NaN   5
U2   2   3  1.0  5.0   3
U3   5   4  3.0  4.0   2
U4   4   2  2.0  5.0   4
U5   3   4  NaN  3.0   5
U6   5   3  5.0  5.0   1
U7   5   5  5.0  4.0   5
U8   1   5  2.0  2.0   4

Predicted rating r_{U1, I4} using Pearson: nan
Predicted rating r_{U5, I2} using Pearson: 0.00


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


In [8]:
# 3.2. Câu hỏi tương tự trên nhưng chuẩn hoá bảng theo dạng center và sau đó dùng độ tương đồng cosine. So sánh các kết quả thu được.
# Chuẩn hoá bảng theo dạng center
centered_matrix = utility_matrix.sub(utility_matrix.mean(axis=1), axis=0)
print("\nCentered Utility Matrix:")
print(centered_matrix)

# Hàm tính độ tương đồng cosine giữa hai user
def cosine_sim(user1, user2):
    common_items = user1.notna() & user2.notna()
    if common_items.sum() == 0:
        return 0
    return cosine_similarity([user1[common_items]], [user2[common_items]])[0][0]

def predict_cosine(user_id, item_id):
    user = centered_matrix.loc[user_id]
    similarities = []
    ratings = []
    
    for other_user_id in centered_matrix.index:
        if other_user_id != user_id and not pd.isna(centered_matrix.loc[other_user_id, item_id]):
            sim = cosine_sim(user, centered_matrix.loc[other_user_id])
            similarities.append(sim)
            ratings.append(utility_matrix.loc[other_user_id, item_id])  # Dùng rating gốc để dự đoán
    
    if sum(np.abs(similarities)) == 0:
        return np.nan  # Không thể dự đoán nếu không có độ tương đồng
    
    predicted_rating = np.dot(similarities, ratings) / sum(np.abs(similarities))
    return predicted_rating

predicted_r_U1_I4_cosine = predict_cosine('U1', 'I4')
predicted_r_U5_I2_cosine = predict_cosine('U5', 'I2')
print(f"\nPredicted rating r_{{U1, I3}} using Cosine: {predicted_r_U1_I4_cosine:.2f}")
print(f"Predicted rating r_{{U4, I1}} using Cosine: {predicted_r_U5_I2_cosine:.2f}")

# So sánh kết quả
print("\nComparison of Predictions:")
print(f"r_{{U1, I3}} - Pearson: {predicted_r_U1_I4:.2f}, Cosine: {predicted_r_U1_I4_cosine:.2f}")
print(f"r_{{U4, I1}} - Pearson: {predicted_r_U5_I2:.2f}, Cosine: {predicted_r_U5_I2_cosine:.2f}")


Centered Utility Matrix:
      I1    I2   I3    I4    I5
U1  0.00 -1.00  0.0   NaN  1.00
U2 -0.80  0.20 -1.8  2.20  0.20
U3  1.40  0.40 -0.6  0.40 -1.60
U4  0.60 -1.40 -1.4  1.60  0.60
U5 -0.75  0.25  NaN -0.75  1.25
U6  1.20 -0.80  1.2  1.20 -2.80
U7  0.20  0.20  0.2 -0.80  0.20
U8 -1.80  2.20 -0.8 -0.80  1.20

Predicted rating r_{U1, I3} using Cosine: -0.14
Predicted rating r_{U4, I1} using Cosine: 0.02

Comparison of Predictions:
r_{U1, I3} - Pearson: nan, Cosine: -0.14
r_{U4, I1} - Pearson: 0.00, Cosine: 0.02


In [9]:
# 3.3 Một người thực hiện dự đoán theo kiểu tuần tự, tức là dự đoán xong missing rating thứ $1$ thì điền rating đó vào bảng hiện tại rồi mới thực hiện tiếp cho missing rating thứ $2$. Anh chị hãy thực hiện cách làm này cho một trong hai ý 1) hoặc 2) và so sánh kết quả thu được. Hỏi cách làm này nói chung sẽ có các ưu nhược điểm gì?
# Sử dụng phương pháp Pearson để dự đoán missing ratings theo kiểu tuần tự
def sequential_predict_pearson(missing_ratings):
    for user_id, item_id in missing_ratings:
        predicted_rating = predict_pearson(user_id, item_id)
        utility_matrix.loc[user_id, item_id] = predicted_rating  # Cập nhật bảng với rating dự đoán
        print(f"Predicted rating r_{{{user_id}, {item_id}}} using Pearson: {predicted_rating:.2f}")
        print(utility_matrix)  # In bảng sau mỗi lần dự đoán
    return utility_matrix

# Danh sách các missing ratings cần dự đoán
missing_ratings = [('U1', 'I4'), ('U5', 'I2')]
print("\nSequential Predictions using Pearson:")
final_matrix = sequential_predict_pearson(missing_ratings)
print("\nFinal Utility Matrix after Sequential Predictions:")
print(final_matrix)


Sequential Predictions using Pearson:
Predicted rating r_{U1, I4} using Pearson: nan
    I1  I2   I3   I4  I5
U1   4   3  4.0  NaN   5
U2   2   3  1.0  5.0   3
U3   5   4  3.0  4.0   2
U4   4   2  2.0  5.0   4
U5   3   4  NaN  3.0   5
U6   5   3  5.0  5.0   1
U7   5   5  5.0  4.0   5
U8   1   5  2.0  2.0   4
Predicted rating r_{U5, I2} using Pearson: 0.00
    I1        I2   I3   I4  I5
U1   4  3.000000  4.0  NaN   5
U2   2  3.000000  1.0  5.0   3
U3   5  4.000000  3.0  4.0   2
U4   4  2.000000  2.0  5.0   4
U5   3  0.003267  NaN  3.0   5
U6   5  3.000000  5.0  5.0   1
U7   5  5.000000  5.0  4.0   5
U8   1  5.000000  2.0  2.0   4

Final Utility Matrix after Sequential Predictions:
    I1        I2   I3   I4  I5
U1   4  3.000000  4.0  NaN   5
U2   2  3.000000  1.0  5.0   3
U3   5  4.000000  3.0  4.0   2
U4   4  2.000000  2.0  5.0   4
U5   3  0.003267  NaN  3.0   5
U6   5  3.000000  5.0  5.0   1
U7   5  5.000000  5.0  4.0   5
U8   1  5.000000  2.0  2.0   4


c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())


Ưu điểm:
- Cập nhật bảng liên tục giúp cải thiện độ chính xác của các dự đoán tiếp theo.
- Sử dụng thông tin mới nhất để dự đoán, có thể phản ánh tốt hơn sở thích của user.
Nhược điểm:
- Tăng thời gian tính toán do phải cập nhật bảng sau mỗi dự đoán.
- Có thể dẫn đến lỗi lan truyền nếu một dự đoán sai ảnh hưởng đến các dự đoán sau đó.

**Bài 4.** (*2.5 điểm*) Sử dụng lại dataset đã cho ở **Bài 1** và chọn một thuật toán *collaborative filtering* đã chuẩn bị sẵn, trong đó ở ý tưởng kNN, ta chọn $k=5$.

1) Lọc và giữ lại các user đã rate cho ít nhất $5$ phim. Từ đó in ra ma trận tương đồng giữa các user đó dựa trên độ tương đồng phù hợp.
2) Hãy nêu một số ưu điểm-nhược điểm cơ bản khi so sánh giữa khuyến nghị theo *user-based* và *item-based*.
3) Duyệt theo từng user và cho biết sẽ khuyến nghị các item nào cho họ.
4) Duyệt theo từng item và cho biết sẽ khuyến nghị item này cho các user nào.

In [10]:
# 4.1. Gợi ý cho user đã nghe ít nhất 5 bài hát
import pandas as pd
import numpy as np

df = pd.read_csv("T:\GK_he khuyen nghi_ca 1\data.csv")
df.columns = ['user_id', 'song_id', 'rating']

# Tính số lượng rating và trung bình rating cho mỗi bài hát
user_stats = df.groupby('user_id').agg(['count']).reset_index()

# Tính user rating it nhất 5 song
df_user = df[df['user_id'].isin(user_stats[user_stats[('rating', 'count')] >= 5]['user_id'])]

print("=== 🧠 USER ĐÃ NGHE ÍT NHẤT 5 BÀI HÁT ===")
print(df_user)

=== 🧠 USER ĐÃ NGHE ÍT NHẤT 5 BÀI HÁT ===
      user_id  song_id  rating
0           3       25       4
1           3        9       3
2           3       15       1
3           3       21       2
4           3        3       2
...       ...      ...     ...
1663      396       16       2
1664      396       14       1
1665      396       18       1
1666      396       13       5
1667      396       23       5

[1181 rows x 3 columns]


4.2 Ưu điểm và nhược điểm của khuyến nghị theo user-based và item-based:
1. User-Based Collaborative Filtering:
   - Ưu điểm:
     + Dễ hiểu và triển khai.
     + Có thể phát hiện các nhóm người dùng có sở thích tương tự.
   - Nhược điểm:
     + Hiệu suất kém khi số lượng người dùng lớn (vấn đề scalability).
     + Khó khăn trong việc xử lý dữ liệu thưa thớt (sparse data).
     + Độ chính xác có thể giảm nếu người dùng mới không có nhiều lịch sử đánh giá.
2. Item-Based Collaborative Filtering:
    - Ưu điểm:
      + Hiệu suất tốt hơn khi số lượng người dùng lớn vì số lượng item thường ít hơn.
      + Ổn định hơn với dữ liệu thưa thớt vì tập hợp các item thường có nhiều đánh giá hơn.
      + Dễ dàng mở rộng và cập nhật khi có thêm item mới.
    - Nhược điểm:
      + Khó khăn trong việc phát hiện các nhóm người dùng có sở thích tương tự.
      + Có thể bỏ lỡ các xu hướng mới nếu không có nhiều đánh giá cho item mới.

In [11]:
# 4.3 Duyệt theo từng user và cho biết sẽ khuyến nghị item nào cho họ.
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Kiểm tra và xử lý các giá trị trùng lặp (tính trung bình rating cho các user-item trùng lặp)
df_user = df_user.groupby(['user_id', 'song_id'], as_index=False)['rating'].mean()

# Chuyển dữ liệu sang ma trận user-item (utility matrix)
utility_matrix = df_user.pivot(index='user_id', columns='song_id', values='rating')

# Điền giá trị NaN bằng 0 để tính độ tương đồng
utility_matrix_filled = utility_matrix.fillna(0)

# Tính ma trận tương đồng giữa các user
user_similarity = cosine_similarity(utility_matrix_filled)

# Hàm khuyến nghị item cho user
def recommend_for_user(user_id, utility_matrix, user_similarity, top_n=3):
    user_ratings = utility_matrix.loc[user_id]
    recommended_items = {}
    
    # Duyệt qua các item mà user chưa xem
    for item_id in utility_matrix.columns:
        if pd.isna(user_ratings[item_id]):  # Nếu user chưa xem item
            weighted_sum = 0
            similarity_sum = 0
            # Duyệt qua các user khác để tính weighted average
            for other_user_id in utility_matrix.index:
                if not pd.isna(utility_matrix.loc[other_user_id, item_id]):  # Nếu user khác đã xem item
                    sim = user_similarity[utility_matrix.index.get_loc(user_id), utility_matrix.index.get_loc(other_user_id)]
                    weighted_sum += sim * utility_matrix.loc[other_user_id, item_id]
                    similarity_sum += abs(sim)
            if similarity_sum > 0:
                recommended_items[item_id] = weighted_sum / similarity_sum
            else:
                recommended_items[item_id] = user_ratings.mean()  # Rating trung bình nếu không có độ tương đồng
    
    # Sắp xếp các item theo độ ưu tiên cao nhất
    recommended_items = {k: v for k, v in sorted(recommended_items.items(), key=lambda item: item[1], reverse=True)}
    
    # Trả về top_n item khuyến nghị
    return dict(list(recommended_items.items())[:top_n])

# Duyệt qua tất cả các user và khuyến nghị các item cho từng user
for user in utility_matrix.index:
    user_recommendations = recommend_for_user(user, utility_matrix, user_similarity, top_n=3)
    if user_recommendations:
        print(f"\nTop 3 Recommendations for {user} (User-based):")
        print(user_recommendations)
    else:
        print(f"\nNo recommendations for {user} (User-based).")



Top 3 Recommendations for 3 (User-based):
{7: 3.4062865515361347, 22: 3.2494516748342774, 24: 3.2055075414266083}

Top 3 Recommendations for 5 (User-based):
{22: 3.3974210852733924, 18: 3.2342780412613505, 1: 3.2093698181736174}

Top 3 Recommendations for 7 (User-based):
{24: 3.3798049832657946, 3: 3.217103649161224, 25: 3.2107912125753018}

Top 3 Recommendations for 10 (User-based):
{3: 3.4107861346607566, 7: 3.2872807793736434, 22: 3.241326817028961}

Top 3 Recommendations for 11 (User-based):
{3: 3.4868335441398, 24: 3.3837109864724315, 10: 3.3275222763750896}

Top 3 Recommendations for 14 (User-based):
{3: 3.4775377768701867, 24: 3.4118888122303366, 10: 3.222021195279661}

Top 3 Recommendations for 15 (User-based):
{7: 3.2626786670502637, 4: 3.2544830256846513, 3: 3.185437398750649}

Top 3 Recommendations for 16 (User-based):
{3: 3.2162826322357763, 24: 3.2130264252193035, 4: 3.2103926229160358}

Top 3 Recommendations for 18 (User-based):
{7: 3.3066569627100724, 1: 3.2025901528035

In [12]:
# 4.4 Duyệt theo từng item và cho biết sẽ khuyến nghị item này cho các user nào
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# 2️⃣ Xử lý trùng lặp (nếu có)
df_user = df_user.groupby(['user_id', 'song_id'], as_index=False)['rating'].mean()

# 3️⃣ Tạo utility matrix (user-item)
utility_matrix = df_user.pivot(index='user_id', columns='song_id', values='rating')

# 4️⃣ Tính ma trận tương đồng giữa các item
item_similarity = cosine_similarity(utility_matrix.T.fillna(0))

# Gắn lại index và columns cho dễ nhìn
item_similarity_df = pd.DataFrame(
    item_similarity, 
    index=utility_matrix.columns, 
    columns=utility_matrix.columns
)
print("Item-Item Similarity Matrix:")
print(item_similarity_df.head())

# 5️⃣ Hàm khuyến nghị user cho từng item
def recommend_for_item(item_id, utility_matrix, item_similarity, top_n=3):
    item_ratings = utility_matrix[item_id]
    recommended_users = {}
    
    for user_id in utility_matrix.index:
        if pd.isna(item_ratings[user_id]):  # Nếu user chưa đánh giá item
            weighted_sum = 0
            similarity_sum = 0
            for other_item_id in utility_matrix.columns:
                if not pd.isna(utility_matrix.loc[user_id, other_item_id]):  # Nếu user đã đánh giá item khác
                    sim = item_similarity[utility_matrix.columns.get_loc(item_id),
                                          utility_matrix.columns.get_loc(other_item_id)]
                    weighted_sum += sim * utility_matrix.loc[user_id, other_item_id]
                    similarity_sum += abs(sim)
            if similarity_sum > 0:
                predicted_rating = weighted_sum / similarity_sum
                recommended_users[user_id] = predicted_rating

    # Lọc top user có điểm dự đoán cao nhất
    recommended_users = dict(sorted(recommended_users.items(), key=lambda x: x[1], reverse=True)[:top_n])
    return recommended_users

# 6️⃣ Duyệt qua từng item và in user được khuyến nghị
for item in utility_matrix.columns[:5]:  # chỉ in 5 item đầu cho gọn
    item_recommendations = recommend_for_item(item, utility_matrix, item_similarity, top_n=3)
    print(f"\nTop 3 users được khuyến nghị cho item {item}:")
    if item_recommendations:
        print(item_recommendations)
    else:
        print("Không có user nào cần khuyến nghị cho item này.")


Item-Item Similarity Matrix:
song_id        1         2         3         4         5         6         7   \
song_id                                                                         
1        1.000000  0.111388  0.182437  0.198735  0.251418  0.211045  0.251666   
2        0.111388  1.000000  0.144682  0.096861  0.110304  0.132405  0.155894   
3        0.182437  0.144682  1.000000  0.178801  0.295629  0.101390  0.280243   
4        0.198735  0.096861  0.178801  1.000000  0.091342  0.236157  0.168181   
5        0.251418  0.110304  0.295629  0.091342  1.000000  0.313943  0.345153   

song_id        8         9         10  ...        16        17        18  \
song_id                                ...                                 
1        0.220210  0.134889  0.178220  ...  0.075969  0.221535  0.331993   
2        0.120856  0.229717  0.238203  ...  0.307069  0.121584  0.239630   
3        0.177652  0.151512  0.136646  ...  0.212635  0.142575  0.095517   
4        0.070569  0.22

**Bài 5.** (*0.5 điểm*) Trong HAI chọn MỘT.

1) Dựa theo **Bài 1**, nếu có một user đã nghe $3$ bài hát nào đó trong $25$ bài hát đã cho thì muốn khuyến nghị top $5$ bài hát mà user này có thể muốn nghe, anh/chị sẽ làm thế nào? Chỉ mô tả ý tưởng dựa trên cách làm ở ý **1.2)** ở trên, không cần code chi tiết.
2) Dựa theo **Bài 4**, về lý thuyết thì hai ma trận mô tả user-item trong kết quả ở **4.3)** và **4.4)** có quan hệ gì với nhau?

5.1 Ý tưởng:
- Xác định 3 bài hát mà user đã nghe trong số 25 bài hát đã cho.
- Sử dụng ma trận tương đồng đã tính ở ý 1.2 để tìm các bài hát tương đồng với 3 bài hát mà user đã nghe.
- Tính điểm tổng hợp cho các bài hát chưa nghe dựa trên độ tương đồng với 3 bài hát đã nghe.
- Sắp xếp các bài hát theo điểm tổng hợp và chọn top 5 bài hát có điểm cao nhất để khuyến nghị cho user.